In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
# Read in data from Jeremy 
df1 = pd.read_excel('../data/raw/2024-09-01 Standard Coverage Model - JR.xlsx', sheet_name= 'Rep Addresses');

# Preview first few rows
df1.head(3)



,Division,Region,Region\nName,Territory,Name,Last Name,* FAX # \nCELL #,Address_1,City,State,Zip,E-mail,Country,Comments,rd?,Regionals to Sort Separate,Notes
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN,NaN
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN,NaN
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN,NaN


In [3]:
# Get dimensions of data
print(df1.shape)

# Make columns more readable
df1 = df1.rename(columns={'Division': 'division',
                          'Region': 'region_code',
                          'Territory' : 'territory',
                          'Name': 'name',
                          'Last Name' : 'last_name',
                          'Region\nName': 'region_name', 
                          ' * FAX # \nCELL #': 'fax/cell',
                         'Address_1' : 'street_address',
                          'City' : 'city',
                          'State' : 'state',
                          'Zip' : 'zip',
                          'E-mail' : 'email',
                          'Country' : 'country',
                          'Comments' : 'comments',
                          'Regionals to Sort Separate' : 'regionals_to_sort_separate',
                          'Notes' : 'notes'
                         })

(112, 17)


In [4]:
# Drop any rows that are completely filled with NA
df1 = df1.dropna(how='all')

# Create full_address feature for geocoding
df1['full_address'] = df1['street_address'].str.strip() + ', ' + df1['city'].str.strip() + ', ' + df1['state'].str.strip()

# Check that columns are renamed successfully 
df1.head(3)

,division,region_code,region_name,territory,name,last_name,fax/cell,street_address,city,state,zip,email,country,comments,rd?,regionals_to_sort_separate,notes,full_address
0,A,H5,Dallas / Memphis,FN,Micah,Abshier,830.377.5353,1610 Jefferson St.,Kerrville,TX,78028,micah.abshier@batesville.com,NaN,NaN,NaN,NaN,NaN,"1610 Jefferson St., Kerrville, TX"
1,A,H2,Charlotte,HB,Todd,Abshire,540.520.5274,3930 Bethlehem Rd.,Boones Mill,VA,24065,todd.abshire@batesville.com,NaN,Address change 6/8/23 per Ashley Brattain HR,NaN,NaN,NaN,"3930 Bethlehem Rd., Boones Mill, VA"
2,A,H4,Batesville,AU,Jim,Adamson,* 515.224.2018\n515.577.5223,33475 Berns Shore Dr.,Adel,IA,50003,james.adamson@batesville.com,NaN,NaN,NaN,NaN,NaN,"33475 Berns Shore Dr., Adel, IA"


In [5]:
# Export to CSV to geocode with external tool
df1.to_csv('../data/processed/rep-addresses.csv')

## Part 2: Read in manually-geocoded addresses 

In [6]:
# Read in geocoded addresses from external tool
rep_addresses_df = pd.read_csv('../data/processed/rep-addresses-geocoded.csv', index_col=0)

In [7]:
rep_addresses_df.columns

# Drop unnecessary columns (if needed)
# rep_addresses_df = rep_addresses_df.drop(['country.1', 'country_code','attribution', 'attribution_license',
#   'attribution_url'], axis=1)

# Drop unnecessary rows (if needed)
# rep_addresses_df = rep_addresses_df[rep_addresses_df.index < 103] 

Index(['division', 'region_code', 'region_name', 'territory', 'name',
       'last_name', 'fax/cell', 'street_address', 'city', 'state', 'zip',
       'email', 'country', 'comments', 'rd?', 'regionals_to_sort_separate',
       'notes', 'full_address', 'lat', 'lon', 'formatted', 'district',
       'name.1', 'housenumber', 'street', 'suburb', 'postcode', 'city.1',
       'county', 'state.1', 'state_code', 'confidence',
       'confidence_city_level', 'confidence_street_level'],
      dtype='object')

In [8]:
# Make columns more readable
rep_addresses_df = rep_addresses_df.rename(columns={'state.1': 'state',
                          'name.1': 'geo_name',
                          'city.1': 'city'
                         })

In [9]:
rep_addresses_df.to_csv('../data/processed/rep-addresses-geocoded-cleaned.csv')

In [10]:
rep_addresses_df['territory'].unique()

array(['FN', 'HB', 'AU', 'R5', 'BB', 'X3', 'REG  ', 'HK', 'X5', 'WB',
       'C8', 'GL', 'FF', '95', 'EJ', 'KY1', 'FB', 'N1', '3', 'X2', '10',
       '39', 'ZV', 'S7', 'DX', 'JP', 'JV', 'JY', 'JF', 'ZT', '85', 'Y1',
       nan, 'WC', '25', 'JD', 'JZ', 'F3', 'S1', 'AX', '33', 'DK', 'GX',
       '51', 'JI', 'JS', 'EE', 'HJ', 'JQ', '17', '4', 'JT', 'ZW', 'JM',
       'DG', 'DW', 'X8', 'GE', 'P5', '2', 'HY', 'GU', 'FV', 'HX', 'HL',
       'JW', 'Y4', 'ZQ', 'ET', 'ZE', 'P2', 'GR', 'EV', 'W7', 'A1', 'S5',
       'WD', 'AV', '57', 'FP', 'EN', 'EM', 'R3', 'Y6', 'JE', 'A7', 'W2',
       'ZD', 'F2', 'GZ', 'HC'], dtype=object)